In [1]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn import preprocessing
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectPercentile, chi2, f_classif, RFECV, mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, brier_score_loss
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from warnings import filterwarnings
from sklearn.model_selection import GridSearchCV
from functools import partial

* read csv
* encoding, missing value handling, oversampling
* models -> voting classifier for improvements
* evaluation
* visualization
* <3

In [2]:
filterwarnings(action='ignore')

In [3]:
train_df = pd.read_csv('lucene-solr/dataset/training.csv')
test_df = pd.read_csv('lucene-solr/dataset/testing.csv')

In [4]:
print('train: ', len(train_df))
print('test: ', len(test_df))

train:  3657
test:  5162


In [5]:
for df in [train_df, test_df]:
    df.dropna(subset=list(df.columns).remove('Bugged'), inplace=True)
    for col in df.columns:
        df[col] = df[col].apply(lambda x: 1 if x is True else 0)

In [6]:
X_train = train_df.drop(['Bugged'], axis=1)
y_train = train_df['Bugged']

X_test = test_df.drop(['Bugged'], axis=1)
y_test = test_df['Bugged']

In [7]:
print('train: ', len(train_df))
print('test: ', len(test_df))

train:  3657
test:  5162


In [8]:
train_df = train_df.dropna()

In [9]:
models = {
        'LinearDiscriminantAnalysis': LinearDiscriminantAnalysis(),
        'QuadraticDiscriminantAnalysis': QuadraticDiscriminantAnalysis(),
        'LogisticRegression': LogisticRegression(),
        'BernoulliNaiveBayes': BernoulliNB(),
        'K-NearestNeighbor': KNeighborsClassifier(),
        'DecisionTree': DecisionTreeClassifier(),
        'RandomForest': RandomForestClassifier(),
        'SupportVectorMachine': SVC(),
        'MultilayerPerceptron': MLPClassifier()
    }

params = {
        'LinearDiscriminantAnalysis': {},
        'QuadraticDiscriminantAnalysis': {},
        'LogisticRegression': {'C': list(np.logspace(-4, 4, 3))},
        'BernoulliNaiveBayes': {},
        'K-NearestNeighbor': {},
        'DecisionTree': {'criterion': ['gini', 'entropy'], },
        'RandomForest': {'n_estimators': [10, 100]},
        'SupportVectorMachine': {'C': [0.1, 100]},
        'MultilayerPerceptron': {'hidden_layer_sizes': [(17, 8, 17)],
                                 'activation': ['tanh', 'relu']}
    }

selection_methods = {
        'chi2_20p': SelectPercentile(chi2, percentile=20),
        'chi2_50p': SelectPercentile(chi2, percentile=50),
        'mutual_info_classif_20p': SelectPercentile(mutual_info_classif, percentile=20),
        'mutual_info_classif_50p': SelectPercentile(mutual_info_classif, percentile=50),
        'f_classif_20': SelectPercentile(f_classif, percentile=20),
        'f_classif_50': SelectPercentile(f_classif, percentile=50),
        'recursive_elimination': RFECV(RandomForestClassifier(), min_features_to_select=3, step=1, cv=5, scoring='f1')
    }


In [87]:
def select(X, y):
    selected_data = pd.DataFrame(columns=list(selection_methods.keys()))
    selected_features = pd.DataFrame(columns=list(selection_methods.keys()))
    features = X.columns
    for method_name, method in selection_methods.items():
        selected_data[method_name] = method.fit_transform(X, y).tolist()
        features_mask = method.get_support()
        selected_features[method_name] = np.array(features)[features_mask].tolist()
        print(np.array(features)[features_mask].tolist())
#     selected_data['all'] = X
#     selected_features['all'] = list(features)
    return selected_features, selected_data

In [88]:
selected_features_train, selected_data_train = select(X_train, y_train)

['UnutilizedAbstraction', 'DeficientEncapsulation', 'InsufficientModularization', 'GodClass', 'ClassDataShouldBePrivate', 'LazyClass', 'SpaghettiCode', 'LargeClass']


ValueError: Length of values (18) does not match length of index (8)

In [12]:
# oversampled_datasets = {method: SMOTE().fit_resample(X_train, y_train) for method, X_train in selected_data.items()}

# selected_features, selected_dataset = select(X_train, y_train)

In [15]:
def get_selected_testing(test_df, selected_features_train):
    features = test_df.columns
    test_y = test_df['Bugged']
    selected_testing_datasets = {
    method: (test_df[test_df.columns.intersection(features)].values, test_y)
    for method, features in selected_features_train.items()
    }
    return selected_testing_datasets

In [16]:
selected_testing_datasets = get_selected_testing(test_df, selected_features_train)

In [17]:
grid_searches = {}

In [18]:
def fit(X, y, cv=5, n_jobs=1, verbose=1, scoring=None, refit=False):
    for key in models.keys():
        model = models[key]
        param = params[key]
        gs = GridSearchCV(model, param, cv=cv, n_jobs=n_jobs, verbose=verbose,
                          scoring=scoring, refit=refit, return_train_score=True)
        gs.fit(X, y)
        grid_searches[key] = gs

In [53]:
def score_summary(sort_by='mean_score'):

    def extract_rows(key: str):
        def get_cv_results(cv, params):
            key = "split{}_test_score".format(cv)
            print(grid_search.cv_results_[key])
            return grid_search.cv_results_[key]

        def row(key, scores, params):
            d = {
                'estimator': key,
                'min_score': min(scores),
                'max_score': max(scores),
                'mean_score': np.mean(scores),
                'std_score': np.std(scores)
            }
            return pd.Series({params, d})
    
        grid_search = grid_searches[key]
        print(grid_search)
        params = grid_search.cv_results_['params']
        get_cv_results_with_params = partial(get_cv_results, params=params)
        scores = np.hstack(list(map(get_cv_results_with_params, range(grid_search.cv))))
        summary = list(map(lambda values:
                           row(key, values[1], values[0]),
                           list(zip(params, scores))))
        return summary
    
    rows = list(itertools.chain.from_iterable(map(extract_rows, grid_searches.keys())))
    df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)
    columns = ['estimator', 'min_score', 'mean_score', 'max_score', 'std_score']
    columns = columns + [c for c in df.columns if c not in columns]
    return df[columns]


In [58]:
def get_summary(X, y):
    fit(X, y)
    return score_summary()

In [59]:
selected_data_train['y'] = y_train

In [61]:
get_summary(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Fitting 5 folds for each of 2 candidates, totalling 10 fits
{'LinearDiscriminantAnalysis': GridSearchCV(cv=5, estimator=LinearDiscriminantAnalysis(), n_jobs=1,
             param_grid={}, refit=False, return_train_score=True, verbose=1), 'QuadraticDiscriminantAnalysis': GridSearchCV(cv=5, estimator=QuadraticDiscriminantAnalysis(), n_jobs=1,
             param_grid={}, refit=False, return_train_score=True, verbose=1), 'LogisticRegression': GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=1,
             param_g

NameError: name 'itertools' is not defined

In [60]:
# summaries = {method: get_summary(data[0], data[1])
#                      for method, data in selected_data_train.items()}

# top_summaries = {method: summary[:n] for method, summary in summaries.items()}

# configurations = {method: list(map(lambda x: x[1].to_dict(),
#                                            top_summary.drop(EstimatorSelectionHelper.get_scores_info(),
#                                                             axis=1)
#                                            .where(pd.notnull(top_summary), None).iterrows()))
#                           for method, top_summary in top_summaries.items()}

# method_names = configurations.keys()
#         scores_dicts = list(map(lambda method_name:
#                                 list(map(lambda configuration:
#                                          calculate_score(method_name,
#                                                          oversampled_training[method_name],
#                                                          selected_testing[method_name],
#                                                          configuration),
#                                          configurations[method_name])), method_names))
#         scores_df = [pd.DataFrame(score) for score in scores_dicts]
#         scores = pd.concat(scores_df)


Fitting 5 folds for each of 1 candidates, totalling 5 fits


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "D:\IUST\4011\Software Architecture\imple\Exploring-Design-smells-for-smell-based-defect-prediction\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\IUST\4011\Software Architecture\imple\Exploring-Design-smells-for-smell-based-defect-prediction\venv\lib\site-packages\sklearn\discriminant_analysis.py", line 575, in fit
    X, y = self._validate_data(
  File "D:\IUST\4011\Software Architecture\imple\Exploring-Design-smells-for-smell-based-defect-prediction\venv\lib\site-packages\sklearn\base.py", line 554, in _validate_data
    X, y =